Alright — here’s a **fully detailed, structured set of notes** for your **DBMS – Transactions Recoverability** topic, covering **every single point** from your transcript and aligned with the stated learning outcomes.

---

## **DBMS – Transactions Recoverability**

### **1. Introduction**

* **Context:**

  * In previous modules, we studied *concurrent transaction execution* and problems like **lost update**, **temporary update**, **dirty read**, etc.
  * **Serializability** ensures that concurrent execution of transactions maintains **isolation** and **consistency**.
  * **Conflict Serializability** is a widely used criterion for correctness of concurrent schedules.
* **Problem:**

  * Even with serializability, **atomicity** and **consistency** can be compromised if **system failures** occur (power outage, hardware crash, software crash).
* **Goal of this topic:**

  * Understand **Recoverability** – the ability to restore the database to a **consistent state** after a failure.
  * Explore **View Serializability** – a weaker form of serializability that allows more concurrent schedules compared to conflict serializability.

---

## **2. Recovery Problem**

### **2.1 What is the Recovery Problem?**

* **Scenario:**

  * Suppose a serial transaction transfers ₹50 from account A to account B.
  * If the system fails after A is updated but before `COMMIT`, the database will have inconsistent data:

    * A’s value decreased.
    * B’s value not yet increased.
* **Need:**

  * Changes made by a failed transaction **must be undone** → rollback to previous consistent state.

---

## **3. Recoverability**

### **3.1 Definition**

A schedule is **recoverable** if:

> For any two transactions **Ti** and **Tj**,
> If **Tj** reads a data item **Q** written by **Ti**,
> then **Ti** must commit **before** **Tj** commits.

### **3.2 Why Recoverability?**

* If **Ti** aborts after **Tj** has read its uncommitted value, then **Tj** must also abort → prevents inconsistent database state.

### **3.3 Example – Non-Recoverable Schedule**

* **T8** writes A → **T9** reads A → **T9 commits** → **T8 aborts**
* Problem:

  * T9 committed with incorrect (uncommitted) data from T8.
  * Database is now **inconsistent**.

---

## **4. Cascading Rollback**

### **4.1 What is Cascading Rollback?**

* When one transaction aborts, all other transactions that read its intermediate results must also be rolled back.
* This can cause a **chain reaction**:

  * T10 writes A → T11 reads A → T12 reads data from T11.
  * If T10 aborts → T11 aborts → T12 aborts.

### **4.2 Problem**

* Cascading rollback can be long and **costly** in terms of performance.

---

## **5. Cascadeless Schedules**

### **5.1 Definition**

A schedule is **cascadeless** if:

> For every pair of transactions **Ti** and **Tj**,
> If **Tj** reads a data item previously written by **Ti**,
> then **Ti** must commit **before** **Tj** reads it.

### **5.2 Benefit**

* Eliminates cascading rollback.
* Improves performance.
* Still guarantees recoverability.

---

## **6. Examples of Recoverability and Cascadelessness**

1. **Non-Recoverable:**

   * T2 commits before T1 commits.
   * If T1 fails, rollback is impossible.
2. **Recoverable but with Cascading Rollback:**

   * T2 commits after T1 commits → rollback possible.
   * If T1 fails, T2 must also rollback.
3. **Cascadeless:**

   * T2 reads from T1 only after T1 commits → no cascading rollback.

---

## **7. SQL Transaction Control**

### **7.1 Transaction Control Language (TCL) Commands**

1. **COMMIT** – Saves changes permanently.
2. **ROLLBACK** – Undoes changes to the start of transaction or a savepoint.
3. **SAVEPOINT** – Marks a point to which a rollback can occur.
4. **SET TRANSACTION** – Sets properties (e.g., `READ ONLY`, `READ WRITE`).

### **7.2 Important Notes**

* TCL commands apply to **INSERT**, **UPDATE**, **DELETE** only.
* DDL operations (`CREATE`, `DROP`) auto-commit.

### **7.3 Examples**

* **COMMIT Example:**

  ```sql
  DELETE FROM students WHERE age = 25;
  COMMIT;
  ```
* **ROLLBACK Example:**

  * After deletion, if `ROLLBACK` is executed, deleted rows are restored.
* **SAVEPOINT Example:**

  ```sql
  SAVEPOINT sp1;
  DELETE FROM table WHERE id = 1;
  SAVEPOINT sp2;
  DELETE FROM table WHERE id = 2;
  ROLLBACK TO sp2; -- Undoes delete of id=2 but keeps delete of id=1
  ```
* **RELEASE SAVEPOINT:**

  * Removes a savepoint from the transaction.

---

## **8. View Serializability**

### **8.1 Motivation**

* **Conflict Serializability** is strict but excludes many safe concurrent schedules.
* **View Serializability** is weaker → allows more schedules.

### **8.2 Definition**

Two schedules **S** and **S′** are **view equivalent** if they satisfy:

1. **Initial Read Condition:**

   * If a transaction reads the **initial value** of an item in S, it must also read the initial value in S′.
2. **Read-Write Condition:**

   * If Ti reads a value of Q produced by Tj in S, then Ti must read the value of Q produced by Tj in S′.
3. **Final Write Condition:**

   * The transaction that performs the **final write** on Q in S must also do so in S′.

A schedule is **view serializable** if it is view equivalent to a **serial schedule**.

### **8.3 Relation to Conflict Serializability**

* All conflict-serializable schedules are also view-serializable.
* View-serializable schedules that are **not** conflict-serializable have **blind writes** (write without reading).

### **8.4 Limitations**

* **Testing** for view serializability is **NP-complete**.
* Precedence graph (easy for conflict serializability) doesn’t work directly.

---

## **9. Example – Checking View Serializability**

### **Given Schedule Analysis Steps:**

1. List possible serial schedules (n! for n transactions).
2. Apply **Final Write Condition** to eliminate schedules.
3. Apply **Initial Read Condition** to further reduce.
4. Apply **Read-Write Condition**.
5. Remaining schedule(s) = equivalent serial schedule(s).

---

## **10. Beyond View Serializability**

* Some schedules are **not** conflict-serializable or view-serializable, yet produce the **same final result** as some serial execution.
* Detecting such cases requires analyzing **actual operations and data values** → more complex, not covered in standard serializability tests.

---

## **11. Key Takeaways**

* **Recoverability** ensures no transaction commits based on uncommitted data.
* **Cascadeless schedules** improve performance and simplify recovery.
* **TCL in SQL** allows control over transaction boundaries and rollback points.
* **View Serializability** is a more permissive alternative to conflict serializability, but harder to test.
* **Blind writes** are the reason some non-conflict-serializable schedules can still be view-serializable.
* Some schedules are serializable in practice but undetectable by read-write based methods.

---

✅ **Learning Outcomes Achieved:**

* Understood what happens if system fails mid-transaction.
* Learned **Recoverability** rules and cascade issues.
* Covered **Conflict vs View Serializability** and why view serializability is needed.
* Applied **SQL transaction control** for recovery management.